In [2]:
import pandas as pd

In [8]:
# 수도권,동부권,서부권,도서지역 결합데이터
tn_activity_consume_his = pd.read_csv('combined_tn_activity_consume_his_활동소비내역.csv',encoding='utf-8',low_memory=False)
tn_activity_his = pd.read_csv('combined_tn_activity_his_활동내역.csv',encoding='utf-8',low_memory=False)
tn_adv_consume_his = pd.read_csv('combined_tn_adv_consume_his_사전소비내역.csv',encoding='utf-8',low_memory=False)
# tn_companion_info = pd.read_csv('combined_tn_companion_info_동반자정보.csv',encoding='utf-8',low_memory=False)
tn_lodge_consume_his = pd.read_csv('combined_tn_lodge_consume_his_숙박소비내역.csv',encoding='utf-8',low_memory=False)
tn_move_his = pd.read_csv('combined_tn_move_his_이동내역.csv',encoding='utf-8',low_memory=False)
tn_mvmn_consume_his = pd.read_csv('combined_tn_mvmn_consume_his_이동수단소비내역.csv',encoding='utf-8',low_memory=False)
tn_travel = pd.read_csv('combined_tn_travel_여행.csv',encoding='utf-8',low_memory=False)
# tn_traveller_master = pd.read_csv('combined_tn_traveller_master_여행객 Master.csv',encoding='utf-8',low_memory=False)
tn_visit_area_info = pd.read_csv('combined_tn_visit_area_info_방문지정보.csv',encoding='utf-8',low_memory=False)
sgg_code = pd.read_csv('tc_sgg_시군구코드.csv',encoding='utf-8',low_memory=False)
poi_data = pd.read_csv('tn_poi_master_POIMaster.csv',encoding='utf-8',low_memory=False)

In [12]:
poi_data.head(50)
poi_data.shape
poi_data.columns

Index(['POI_ID', 'POI_NM', 'BRNO', 'SGG_CD', 'ROAD_NM_ADDR', 'LOTNO_ADDR',
       'ASORT_LCLASDC', 'ASORT_MLSFCDC', 'ASORT_SDASDC', 'X_COORD', 'Y_COORD',
       'ROAD_NM_CD', 'LOTNO_CD'],
      dtype='object')

In [ ]:
# #중복 TRAVEL_ID제거

# tn_activity_consume_his = tn_activity_consume_his.drop_duplicates(subset='TRAVEL_ID')
# tn_activity_his = tn_activity_his.drop_duplicates(subset='TRAVEL_ID')
# tn_adv_consume_his = tn_adv_consume_his.drop_duplicates(subset='TRAVEL_ID')
# tn_companion_info = tn_companion_info.drop_duplicates(subset='TRAVEL_ID')
# tn_lodge_consume_his = tn_lodge_consume_his.drop_duplicates(subset='TRAVEL_ID')
# tn_move_his = tn_move_his.drop_duplicates(subset='TRAVEL_ID')
# # tn_mvmn_consume_his = tn_mvmn_consume_his.drop_duplicates(subset='TRAVEL_ID')
# tn_traveller_master = tn_traveller_master.drop_duplicates(subset='TRAVEL_ID')
# tn_visit_area_info = tn_visit_area_info.drop_duplicates(subset='TRAVEL_ID')

In [ ]:
import pandas as pd

def merge_dataframes_with_deduplication(dataframes, dataframe_names, output_file, chunk_size=None):
    """
    여러 데이터프레임을 TRAVEL_ID를 기준으로 outer join하며,
    중복된 열의 내용을 비교 후, 모든 값이 동일할 때만 중복 열을 제거합니다.
    결과는 중간 파일로 저장하여 메모리 사용량을 줄입니다.
    """
    if len(dataframes) != len(dataframe_names):
        raise ValueError("데이터프레임 리스트와 이름 리스트의 길이가 같아야 합니다.")

    def process_chunk(result, df, suffix):
        # Outer join 수행
        result = pd.merge(result, df, on='TRAVEL_ID', how='outer', suffixes=('', suffix))
        
        # 중복 열 처리
        duplicate_columns = [col for col in result.columns if suffix in col]
        for col in duplicate_columns:
            original_col = col.replace(suffix, '')
            if original_col in result.columns:
                # 중복 열과 원본 열의 값이 모두 동일한 경우 중복 열 제거
                if result[original_col].equals(result[col]):
                    result.drop(columns=[col], inplace=True)
                else:
                    print(f"Warning: 열 {col}과 {original_col}의 값이 다릅니다. {col} 열을 보존합니다.")
            else:
                # 원본 열이 없으면 중복 열 이름에서 suffix 제거
                result.rename(columns={col: original_col}, inplace=True)
        
        return result

    # 초기 데이터프레임 설정
    result = dataframes[0]

    # 중간 저장 파일 초기화
    result.to_csv(output_file, index=False, encoding="utf-8", mode="w")

    for i, df in enumerate(dataframes[1:], start=1):
        suffix = f"_{dataframe_names[i]}"
        temp_file = f"temp_{i}.csv"

        if chunk_size is None:
            # 전체 병합
            result = process_chunk(result, df, suffix)
        else:
            # 청크 단위 병합
            if isinstance(df, str):  # 데이터프레임 대신 파일 경로일 경우
                for chunk in pd.read_csv(df, chunksize=chunk_size):
                    result = pd.read_csv(output_file)  # 기존 결과 읽기
                    result = process_chunk(result, chunk, suffix)
                    # 중간 결과 저장
                    result.to_csv(output_file, index=False, encoding="utf-8", mode="w")
            else:
                result = process_chunk(result, df, suffix)

    return pd.read_csv(output_file)  # 최종 결과 로드

# 데이터를 나눠 그룹으로 병합
group1 = [tn_activity_consume_his, tn_activity_his, tn_adv_consume_his]
group2 = [tn_lodge_consume_his, tn_move_his, tn_mvmn_consume_his]
group3 = [tn_visit_area_info, tn_travel]

chunk_size = 10_000  # 청크 크기를 줄임

# 그룹 병합 결과 저장 경로 설정
output_group1 = "merged_group1.csv"
output_group2 = "merged_group2.csv"
output_group3 = "merged_group3.csv"

merged_group1 = merge_dataframes_with_deduplication(group1, ["activity_consume", "activity_his", "adv_consume"], output_group1, chunk_size=chunk_size)
merged_group2 = merge_dataframes_with_deduplication(group2, ["lodge_consume", "move_his", "mvmn_consume"], output_group2, chunk_size=chunk_size)
merged_group3 = merge_dataframes_with_deduplication(group3, ["visit_area_info", "travel"], output_group3, chunk_size=chunk_size)

# 그룹 간 병합
final_output = "final_merged.csv"
final_merged = merge_dataframes_with_deduplication(
    [merged_group1, merged_group2, merged_group3],
    ["group1", "group2", "group3"],
    final_output,
    chunk_size=chunk_size
)

# 최종 결과 확인
print(final_merged.head())
print(final_merged.shape)


C:\Users\user\AppData\Local\Temp\ipykernel_14756\549510925.py:56: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(output_file)  # 최종 결과 로드


   TRAVEL_ID  VISIT_AREA_ID  ACTIVITY_TYPE_CD  ACTIVITY_TYPE_SEQ  \
0  a_a004739     2210020002                 3                  0   
1  a_a002397     2209170002                 3                  1   
2  a_a017904     2211110007                 1                  0   
3  a_a018111     2211130004                 1                  0   
4  a_a005012     2210120010                 1                  0   

   CONSUME_HIS_SEQ  CONSUME_HIS_SNO  PAYMENT_NUM          BRNO       STORE_NM  \
0                0                1            2           NaN        아침고요수목원   
1                1                0            1           NaN          에버랜드    
2                0                1            2  5.918702e+09           금별맥주   
3                0                1            2  3.328101e+09        주식회사필모어   
4                0                1            1  6.448101e+09  박가네푸드시스템 주식회사   

              ROAD_NM_ADDR               LOTNO_ADDR  ROAD_NM_CD      LOTNO_CD  \
0       경기 가평군 상면 수목원로 

In [13]:
print(final_merged.columns)

Index(['TRAVEL_ID', 'VISIT_AREA_ID', 'ACTIVITY_TYPE_CD', 'ACTIVITY_TYPE_SEQ',
       'CONSUME_HIS_SEQ', 'CONSUME_HIS_SNO', 'PAYMENT_NUM', 'BRNO', 'STORE_NM',
       'ROAD_NM_ADDR', 'LOTNO_ADDR', 'ROAD_NM_CD', 'LOTNO_CD', 'PAYMENT_DT',
       'PAYMENT_MTHD_SE', 'PAYMENT_AMT_WON', 'PAYMENT_ETC', 'SGG_CD'],
      dtype='object')


In [12]:
final_merged = pd.read_csv('final_merged.csv', encoding='utf-8')
POI_data = pd.read_csv('tn_poi_master_POIMaster.csv', encoding='utf-8')

final_merged['SGG_CD'] = final_merged['SGG_CD'].astype(str)
POI_data['SGG_CD'] = POI_data['SGG_CD'].astype(str)

# 'SGG_CD'를 기준 병합
merged_data = pd.merge(final_merged, POI_data, on='SGG_CD', how='outer', suffixes=('', '_POI'))

# 결측치 확인
missing_summary = merged_data.isnull().sum() / len(merged_data) * 100
print("결측치 비율:\n", missing_summary)

# 병합된 데이터 저장
merged_data.to_csv('merged_data_sgg_cd.csv', index=False, encoding='utf-8')
print("SGG_CD 기준으로 병합된 데이터 저장 완료")

C:\Users\user\AppData\Local\Temp\ipykernel_15452\2561976304.py:2: DtypeWarning: Columns (3,4,12) have mixed types. Specify dtype option on import or set low_memory=False.
  POI_data = pd.read_csv('tn_poi_master_POIMaster.csv', encoding='utf-8')


결측치 비율:
 TRAVEL_ID            98.849432
VISIT_AREA_ID        98.849432
ACTIVITY_TYPE_CD     98.849432
ACTIVITY_TYPE_SEQ    98.849432
CONSUME_HIS_SEQ      98.849432
CONSUME_HIS_SNO      98.849432
PAYMENT_NUM          98.849432
BRNO                 99.115745
STORE_NM             98.859937
ROAD_NM_ADDR         98.977249
LOTNO_ADDR           99.293021
ROAD_NM_CD           99.071983
LOTNO_CD             99.071983
PAYMENT_DT           99.036616
PAYMENT_MTHD_SE      98.854983
PAYMENT_AMT_WON      98.849804
PAYMENT_ETC          99.294269
SGG_CD                0.000000
POI_ID                1.150568
POI_NM                1.150634
BRNO_POI              1.150568
ROAD_NM_ADDR_POI     15.394293
LOTNO_ADDR_POI        1.150568
ASORT_LCLASDC        87.585680
ASORT_MLSFCDC        87.585680
ASORT_SDASDC         87.585680
X_COORD               1.150568
Y_COORD               1.150568
ROAD_NM_CD_POI        1.150568
LOTNO_CD_POI          1.150568
dtype: float64
SGG_CD 기준으로 병합된 데이터 저장 완료


In [15]:
# 고유값 및 개수 확인
sgg_cd_values = merged_data['SGG_CD'].value_counts()

# 고유값 리스트와 개수 출력
print("Unique values and counts in SGG_CD:")
print(sgg_cd_values)


Unique values and counts in SGG_CD:
SGG_CD
                573029
nan              73233
1168010100       24026
1165010800       23324
1162010200       23178
                 ...  
4148010700.0         1
4671037000           1
4777025023.0         1
5013025930.0         1
3014011700.0         1
Name: count, Length: 21279, dtype: int64


In [ ]:
import pandas as pd

# 데이터 로드
tc_sgg_code = pd.read_csv("./tc_sgg_시군구코드.csv")

# SGG_CD1과 SIDO_NM의 매핑 확인
sgg_sido_mapping = tc_sgg_code[['SGG_CD1', 'SIDO_NM']].drop_duplicates()

# 결과 출력
print(sgg_sido_mapping)



      SGG_CD1  SIDO_NM
0          11    서울특별시
493        26    부산광역시
753        27    대구광역시
1025       28    인천광역시
1201       29    광주광역시
1409       30    대전광역시
1592       31    울산광역시
1744       36  세종특별자치시
1779       41      경기도
2925       42      강원도
3540       43     충청북도
4014       44     충청남도
4601       45     전라북도
5194       46     전라남도
6089       47     경상북도
7117       48     경상남도
7902       50  제주특별자치도


In [27]:
import pandas as pd

# 데이터 로드
tc_sgg_code = pd.read_csv("./tc_sgg_시군구코드.csv")
merged_data = pd.read_csv("merged_data_sgg_cd.csv", encoding="utf-8")

# SGG_CD 열의 값을 문자열로 변환 후 불필요한 공백 제거
merged_data['SGG_CD'] = merged_data['SGG_CD'].astype(str).str.strip()

# 공백 또는 'nan'을 0으로 대체
merged_data['SGG_CD'] = merged_data['SGG_CD'].replace({'': '0', 'nan': '0'}).fillna('0')

# 소수점 제거 후 정수형 변환
merged_data['SGG_CD'] = merged_data['SGG_CD'].str.split('.').str[0].astype(int)

# 지역 매핑 딕셔너리 생성
region_mapping = tc_sgg_code[['SGG_CD1', 'SIDO_NM']].drop_duplicates().set_index('SGG_CD1')['SIDO_NM'].to_dict()

# SGG_CD에서 앞 2자리 추출하여 지역 매핑
merged_data['region'] = merged_data['SGG_CD'].astype(str).str[:2].astype(int).map(region_mapping)

# 결과 확인
print(merged_data[['SGG_CD', 'region']].head())

C:\Users\user\AppData\Local\Temp\ipykernel_15452\378347892.py:5: DtypeWarning: Columns (0,8,9,10,13,16,17,18,19,20,21,22,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv("merged_data_sgg_cd.csv", encoding="utf-8")


   SGG_CD region
0       0    NaN
1       0    NaN
2       0    NaN
3       0    NaN
4       0    NaN


In [2]:
# 각 지역별 데이터 건수 확인
region_distribution = merged_data['region'].value_counts()

# 결과 출력
print(region_distribution)

merged_data.to_csv("mapped_merged_data.csv", index=False, encoding="utf-8")



NameError: name 'merged_data' is not defined

In [3]:
from matplotlib import rc

import matplotlib.pyplot as plt
import pandas as pd

merged_data = pd.read_csv("./mapped_merged_data.csv")


# 각 열의 데이터 개수 확인
column_counts = merged_data.count()
print(column_counts)

C:\Users\user\AppData\Local\Temp\ipykernel_26980\2738140736.py:6: DtypeWarning: Columns (0,8,9,10,13,16,18,19,20,21,22,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv("./mapped_merged_data.csv")


TRAVEL_ID              86632
VISIT_AREA_ID          86632
ACTIVITY_TYPE_CD       86632
ACTIVITY_TYPE_SEQ      86632
CONSUME_HIS_SEQ        86632
CONSUME_HIS_SNO        86632
PAYMENT_NUM            86632
BRNO                   66580
STORE_NM               85841
ROAD_NM_ADDR           77008
LOTNO_ADDR             53232
ROAD_NM_CD             69875
LOTNO_CD               69875
PAYMENT_DT             72538
PAYMENT_MTHD_SE        86214
PAYMENT_AMT_WON        86604
PAYMENT_ETC            53138
SGG_CD               7529498
POI_ID               7442866
POI_NM               7442861
BRNO_POI             7442866
ROAD_NM_ADDR_POI     6370385
LOTNO_ADDR_POI       7442866
ASORT_LCLASDC         934736
ASORT_MLSFCDC         934736
ASORT_SDASDC          934736
X_COORD              7442866
Y_COORD              7442866
ROAD_NM_CD_POI       7442866
LOTNO_CD_POI         7442866
region               6883236
dtype: int64
